# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
#import os
from sqlalchemy import create_engine
import sys
import pickle
import time

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

from scipy.stats import gmean

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e193690\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\e193690\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\e193690\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
database_filepath = './data/DisasterResponse.db'
engine = create_engine('sqlite:///' + database_filepath)
table_name = 'Disaster_response'
df = pd.read_sql_table(table_name,engine)
pd.set_option(('display.max_columns'), 100)
pd.set_option(('display.max_colwidth'), 20)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update -...,Un front froid s...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane...,Cyclone nan fini...,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for some...,"Patnm, di Maryan...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leoga...,UN reports Leoga...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,says: west side ...,facade ouest d H...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df.shape

(26216, 40)

In [5]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
#finding columns that has values more than 1
temp = df.describe().loc['max']>1
df.describe().columns[temp].values

array(['id', 'related'], dtype=object)

In [7]:
#Check for distinct counts of each values of related column
df['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [8]:
# Since the related column has values either 1 or 2 and has 188 values stored as 2,
#hence before we proceed lets replace these values with 1 to consder it as a valid response.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [9]:
#Notice values with 2 are replaced with 1
df['related'].value_counts()

1    20094
0     6122
Name: related, dtype: int64

In [10]:
#Findout the columns that are having only one unique value stored in the dataframe
df.columns[df.nunique() == 1]

Index(['child_alone'], dtype='object')

In [11]:
#Check for distinct counts of each values of child_alone column
df.child_alone.value_counts()

0    26216
Name: child_alone, dtype: int64

In [12]:
#Lets drop the child_alone column since it has all zero values
df.drop(['child_alone'],axis=1, inplace = True)

In [13]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.iloc[:,4:]

In [14]:
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_colwidth', 20)
print('Feature Variable:-------------------------------- \n', X.head(3), '\n', '\n*********************************************************', '\n', 'Target Variables:-------------------------------- \n',  y.head(3))
pd.set_option('display.max_columns', 100)

Feature Variable:-------------------------------- 
 0    Weather update -...
1    Is the Hurricane...
2    Looking for some...
Name: message, dtype: object 
 
********************************************************* 
 Target Variables:-------------------------------- 
    related  request  ...  other_weather  direct_report
0        1        0  ...              0              0
1        1        0  ...              0              0
2        1        0  ...              0              0

[3 rows x 35 columns]


### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    '''
    Tokenize the input string
    
    Input:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the input text message
    '''
    
    # Replace all urls with a default urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # find all urls from the input text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace the url with the default url placeholder string
    for url in detected_urls:
        text = text.replace(url, url_place_holder_string)

    # Extract word tokens from the input text
    tokens = word_tokenize(text)
    
    #Lemmatize to remove different inflected forms of a word so they can be analysed as a single item
    final_tokens = [WordNetLemmatizer().lemmatize(w).lower().strip() for w in tokens]
    return final_tokens

In [16]:
#Testing the tokenize function
text = 'https://www.google.com Hi, This is for testing'
tokenize(text)

['urlplaceholder', 'hi', ',', 'this', 'is', 'for', 'testing']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
# Building the Machine learning pipeline 
def model_pipeline():

    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf_transformer', TfidfTransformer())
                ]))

            ])),

            ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
        ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
# Split the data into train and test sets and train the pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y)

##Added time functions to check how long it takes to train the pipeline
start = time.time()
print('Building and Training of model Started at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start)))

pipeline_model = model_pipeline().fit(X_train, y_train)

end = time.time()
print('Building and Training of model completed at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end)))
print('Building and Training of model took {} seconds'.format(round((end - start), 2)))

Building and Training of model Started at: 2021-03-12 16:18:59
Building and Training of model completed at: 2021-03-12 16:20:32
Building and Training of model took 92.88 seconds


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred_train = pipeline_model.predict(X_train)
y_pred_test = pipeline_model.predict(X_test)

In [20]:
test_text = ['We have lost few of our family members, need some shelters to stay, we are running out of money and homeless, need your urgent help.']
test_result = pipeline_model.predict(test_text)
print("Input test string to test: ", test_text)
print("Predicted results are:",y_train.columns.values[(test_result.flatten()==1)])

Input test string to test:  ['We have lost few of our family members, need some shelters to stay, we are running out of money and homeless, need your urgent help.']
Predicted results are: ['related' 'request' 'aid_related' 'shelter' 'money' 'refugees'
 'direct_report']


In [23]:
# Print classification report on test data
print('\n', classification_report(y_test.values, y_pred_test, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.82      0.94      0.88      5020
               request       0.77      0.53      0.63      1106
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.59      0.66      2721
          medical_help       0.57      0.28      0.37       503
      medical_products       0.60      0.34      0.43       324
     search_and_rescue       0.54      0.16      0.24       190
              security       0.23      0.05      0.08       118
              military       0.68      0.34      0.45       218
                 water       0.73      0.70      0.71       412
                  food       0.80      0.68      0.74       746
               shelter       0.79      0.54      0.64       578
              clothing       0.72      0.48      0.58        91
                 money       0.51      0.32      0.39       135
        missing_people       0.50    

In [24]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15074
               request       0.79      0.56      0.65      3368
                 offer       0.38      0.06      0.10        87
           aid_related       0.77      0.60      0.68      8139
          medical_help       0.66      0.30      0.41      1581
      medical_products       0.73      0.36      0.48       989
     search_and_rescue       0.68      0.21      0.32       534
              security       0.49      0.10      0.17       353
              military       0.68      0.35      0.46       642
                 water       0.77      0.67      0.72      1260
                  food       0.82      0.70      0.76      2177
               shelter       0.80      0.55      0.65      1736
              clothing       0.75      0.47      0.58       314
                 money       0.66      0.34      0.45       469
        missing_people       0.67    

### 6. Improve your model
Use grid search to find better parameters. 

##### Create a performance metric function to define in Grid Search scoring target

In [25]:
# Create a performance metric function to define in Grid Search scoring target
def multioutput_fscore(y_true,y_pred,beta=1):
    """    
    This is a performance metric function to define Grid Search scoring target   
    It is a kind of geometric mean of the fbeta_score, computed on each label.

    The objective here is to avoid issues when dealing with multi-class/multi-label imbalanced cases.
             
    Inputs:
        y_true -> List of labels
        y_prod -> List of predictions
        beta -> Beta value to be used to calculate fscore metric
    
    Output:
        f1score -> Calculation geometric mean of fscore
    """
    
    # If y predictions is a dataframe, then extract and store the values
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    
    # If y actuals is a dataframe, then extract and store the values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    
    f1score_list = []
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        f1score_list.append(score)
        
    f1score = np.asarray(f1score_list)
    f1score = f1score[f1score<1]
    
    # Get the geometric mean of f1score
    f1score = gmean(f1score)
    return f1score

In [26]:
parameters = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

# Split the data into train and test sets and train the pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y)

scorer = make_scorer(multioutput_fscore, greater_is_better = True)

start = time.time()
print('Building and Training of model using GridSearchCV Started at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start)))

# Fit and train the pipeline using grid search cross validator
cv = GridSearchCV(model_pipeline(), param_grid=parameters, scoring=scorer, n_jobs=-1)
cv.fit(X_train, y_train)

end = time.time()
print('Building and Training of model using GridSearchCV Completed at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end)))
print('Building and Training of model using GridSearch took {} seconds'.format(round((end - start), 2)))

Building and Training of model using GridSearchCV Started at: 2021-03-12 16:21:25
Building and Training of model using GridSearchCV Completed at: 2021-03-12 16:28:01
Building and Training of model using GridSearch took 395.37 seconds


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
y_pred_test = cv.predict(X_test)
y_pred_train = cv.predict(X_train)

# Print classification report on test data
print('\n',classification_report(y_test.values, y_pred_test, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.87      5006
               request       0.56      0.37      0.45      1085
                 offer       0.00      0.00      0.00        35
           aid_related       0.79      0.18      0.30      2697
          medical_help       0.59      0.12      0.20       514
      medical_products       1.00      0.01      0.02       313
     search_and_rescue       0.60      0.17      0.26       195
              security       0.00      0.00      0.00       119
              military       0.46      0.12      0.19       231
                 water       0.55      0.83      0.66       424
                  food       0.75      0.67      0.71       697
               shelter       0.83      0.29      0.43       557
              clothing       0.71      0.26      0.38       117
                 money       0.00      0.00      0.00       152
        missing_people       0.60    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.77      1.00      0.87     15088
               request       0.62      0.39      0.48      3389
                 offer       0.00      0.00      0.00        83
           aid_related       0.81      0.19      0.31      8163
          medical_help       0.65      0.11      0.19      1570
      medical_products       0.67      0.00      0.01      1000
     search_and_rescue       0.64      0.20      0.30       529
              security       0.00      0.00      0.00       352
              military       0.54      0.15      0.23       629
                 water       0.59      0.85      0.70      1248
                  food       0.78      0.68      0.73      2226
               shelter       0.84      0.30      0.44      1757
              clothing       0.76      0.34      0.47       288
                 money       1.00      0.01      0.02       452
        missing_people       0.70    

In [30]:
#Find f1 score and Overacl accuracy of model_pipeline using GridsearchCV
multioutput_f1_1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy_1 = (y_pred_test == y_test).mean().mean()

print('F1 score (custom definition) {0:.2f}%'.format(multioutput_f1_1*100))
print('Average overall accuracy {0:.2f}%'.format(overall_accuracy_1*100))

F1 score (custom definition) 91.04%
Average overall accuracy 93.42%


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Build a custom transformer to extract the starting verb of a sentence using our improved pipeline

In [31]:
#Build a custom transformer to extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class    
    It extract the starting verb of a sentence and creates a new feature for the ML classifier
    
    """
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

#### Creating an improved pipeline

In [32]:
def improved_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

#### Build and Train the improved pipeline

In [33]:
#Build and Train the improved pipeline which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
start = time.time()
print('Building and Training of Improved Pipeline Started at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start)))

pipeline_improved_model = improved_pipeline().fit(X_train, y_train)

end = time.time()
print('Building and Training of Improved Pipeline Completed at::', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end)))
print('Building and Training of Improved Pipeline took {} Seconds'.format(round((end - start), 2)))

Building and Training of Improved Pipeline Started at: 2021-03-12 16:28:19
Building and Training of Improved Pipeline Completed at:: 2021-03-12 16:30:24
Building and Training of Improved Pipeline took 124.68 Seconds


In [35]:
y_pred_train1 = pipeline_improved_model.predict(X_train)
y_pred_test1 = pipeline_improved_model.predict(X_test)

# Print classification report on test data
print('\n',classification_report(y_test.values, y_pred_test1, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88      5038
               request       0.80      0.53      0.64      1149
                 offer       0.25      0.03      0.06        32
           aid_related       0.76      0.59      0.67      2753
          medical_help       0.59      0.27      0.37       505
      medical_products       0.54      0.30      0.38       308
     search_and_rescue       0.51      0.25      0.33       169
              security       0.21      0.03      0.06       115
              military       0.63      0.39      0.48       216
                 water       0.75      0.61      0.67       422
                  food       0.82      0.68      0.75       735
               shelter       0.77      0.56      0.64       585
              clothing       0.70      0.40      0.51        94
                 money       0.54      0.29      0.38       148
        missing_people       0.57    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_pred_train1, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15056
               request       0.79      0.56      0.65      3325
                 offer       0.32      0.08      0.13        86
           aid_related       0.77      0.60      0.68      8107
          medical_help       0.65      0.29      0.40      1579
      medical_products       0.72      0.35      0.48      1005
     search_and_rescue       0.69      0.25      0.37       555
              security       0.50      0.08      0.14       356
              military       0.67      0.35      0.46       644
                 water       0.78      0.65      0.71      1250
                  food       0.82      0.68      0.75      2188
               shelter       0.79      0.56      0.65      1729
              clothing       0.77      0.52      0.62       311
                 money       0.63      0.35      0.45       456
        missing_people       0.70    

In [37]:
#Find the F1 Score and accuracy for the improved pipeline
multioutput_f1_2 = multioutput_fscore(y_test,y_pred_test1, beta = 1)
overall_accuracy_2 = (y_pred_test1 == y_test).mean().mean()

print('F1 score (custom definition) {0:.2f}%'.format(multioutput_f1_2*100))
print('Average overall accuracy {0:.2f}%'.format(overall_accuracy_2*100))

F1 score (custom definition) 93.80%
Average overall accuracy 94.69%


### Extra
#### Create a Pipeline using RandomForestClassifier

In [38]:
# Building the Machine learning pipeline with RandomForestClassifier 
def model_pipeline_rf():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                          ('tfidf', TfidfTransformer()),
                          ('clf', MultiOutputClassifier(RandomForestClassifier()))])
        
    return pipeline

In [39]:
# Split the data into train and test sets and train the pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y)

start = time.time()
print('Building and Training of Pipeline with RandomForestClassifier Started at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start)))

pipeline_model_rf = model_pipeline_rf().fit(X_train, y_train)

end = time.time()
print('Building and Training of Pipeline with RandomForestClassifier Completed at:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end)))
print('Building and Training of Pipeline with RandomForestClassifier took {} seconds'.format(round((end - start), 2)))

Building and Training of Pipeline with RandomForestClassifier Started at: 2021-03-12 16:32:20
Building and Training of Pipeline with RandomForestClassifier Completed at: 2021-03-12 16:37:17
Building and Training of Pipeline with RandomForestClassifier took 297.6 seconds


In [40]:
y_pred_train2 = pipeline_model_rf.predict(X_train)
y_pred_test2 = pipeline_model_rf.predict(X_test)

In [41]:
#Find the F1 Score and accuracy for the improved pipeline
multioutput_f1_3 = multioutput_fscore(y_test,y_pred_test2, beta = 1)
overall_accuracy_3 = (y_pred_test2 == y_test).mean().mean()

print('F1 score (custom definition) {0:.2f}%'.format(multioutput_f1_3*100))
print('Average overall accuracy {0:.2f}%'.format(overall_accuracy_3*100))

F1 score (custom definition) 92.94%
Average overall accuracy 94.54%


### 9. Export your model as a pickle file

In [42]:
#Save the model as pickle file
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.